### 참고
- http://yonghee.io/bert_binary_classification_naver/

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.3MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 45.9MB/s 
     |████████████████████████████████| 901kB 48.9MB/s 


In [3]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import random
import time
import datetime

In [4]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


### example source

In [5]:
!git clone https://github.com/e9t/nsmc.git 

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 21.58 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [6]:
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [13]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


### my source

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/자연어처리/dataset.csv')
df = df.dropna().reset_index()
df.head()

,index,gender,name,accords,review
0,0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ..."
1,1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\nI have he...
2,2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...
3,3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...
4,4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\nLove the scent...\...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    75000 non-null  int64 
 1   gender   75000 non-null  object
 2   name     75000 non-null  object
 3   accords  75000 non-null  object
 4   review   75000 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.9+ MB


In [7]:
df['label'] = 0
df.loc[df['gender'] == 'female', 'label'] = 0
df.loc[df['gender'] == 'unisex', 'label'] = 1
df.loc[df['gender'] == 'male', 'label'] = 2
df.head()

,index,gender,name,accords,review,label
0,0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...",0
1,1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\nI have he...,0
2,2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,0
3,3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,0
4,4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\nLove the scent...\...,0


In [8]:
df.groupby('label').count()['review']

label
0    32958
1    14547
2    27495
Name: review, dtype: int64

In [23]:
shuffle = df.sample(frac=1)
# train = shuffle[0:60000].reset_index(drop=True)
# test = shuffle[60000:75000].reset_index(drop=True)

train = shuffle[0:15000].reset_index(drop=True)
test = shuffle[15000:20000].reset_index(drop=True)

In [24]:
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.review]
document_bert[:5]

["[CLS] There were couple times in my life when I felt stressful and every perfume was too much. Only citrus perfumes similar to Light Blue could be worn. For example, I remember when I moved to UK, it was summer and really humid and warm compared to dry and windy chilly summers in my country. Every perfume I have enjoyed before, made me sick. Armani Code, Versace Crystal Noir, C.Klein Euphoria made my stomach turn upside down. I couldn't wear anything but Light Blue because it practically was like citrus deodorant smell, very close to skin and refreshing. [SEP]",
 "[CLS] Beautiful summer scent. I mostly get the herbal (rosemary/ sage) notes - very reminiscent of summer on a sea coast, but in a different way than the original - I don't find them that similar. This is more herbal/ less watery in comparison to the original.\nAwesome performance for a summer scent - lasts easily 8 hours with a decent projection. [SEP]",
 '[CLS] For me is an awfull perfume! Is very heavy and horrible smell

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])

['[CLS]', 'there', 'were', 'couple', 'times', 'in', 'my', 'life', 'when', 'i', 'felt', 'stress', '##ful', 'and', 'every', 'per', '##fum', '##e', 'was', 'too', 'much', '.', 'only', 'cit', '##rus', 'per', '##fum', '##es', 'similar', 'to', 'light', 'blue', 'could', 'be', 'worn', '.', 'for', 'example', ',', 'i', 'remember', 'when', 'i', 'moved', 'to', 'uk', ',', 'it', 'was', 'summer', 'and', 'really', 'hu', '##mid', 'and', 'warm', 'compared', 'to', 'dry', 'and', 'wind', '##y', 'chi', '##lly', 'summer', '##s', 'in', 'my', 'country', '.', 'every', 'per', '##fum', '##e', 'i', 'have', 'enjoyed', 'before', ',', 'made', 'me', 'sick', '.', 'arma', '##ni', 'code', ',', 'vers', '##ace', 'crystal', 'noir', ',', 'c', '.', 'klein', 'eu', '##ph', '##oria', 'made', 'my', 'sto', '##mach', 'turn', 'ups', '##ide', 'down', '.', 'i', 'couldn', "'", 't', 'wear', 'anything', 'but', 'light', 'blue', 'because', 'it', 'practical', '##ly', 'was', 'like', 'cit', '##rus', 'deo', '##dora', '##nt', 'sm', '##ell', ',',

In [26]:
MAX_LEN = 512
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([   101,  11155,  10309,  20969,  13465,  10106,  15127,  12103,
        10841,    177,  24666,  39608,  14446,  10111,  14234,  10178,
       101851,  10112,  10134,  16683,  13172,    119,  10893,  49409,
        13923,  10178, 101851,  10171,  13213,  10114,  15765,  23254,
        12174,  10347,  32091,    119,  10142,  14351,    117,    177,
        93161,  10841,    177,  13059,  10114,  16803,    117,  10271,
        10134,  17995,  10111,  30181,  26506,  59881,  10111,  50089,
        25626,  10114,  36796,  10111,  31346,  10157,  14325,  17903,
        17995,  10107,  10106,  15127,  12723,    119,  14234,  10178,
       101851,  10112,    177,  10529,  52072,  11360,    117,  11019,
        10911, 101003,    119,  28758,  10342,  16964,    117,  12576,
        19929, 104905,  25897,    117,    171,    119,  29185,  14444,
        28088,  25584,  11019,  15127,  47264,  68388,  18923, 107717,
        13315,  12935,    119,    177, 107761,    112,    188,  49866,
      

In [27]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [28]:
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['label'].values, random_state=1, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=1, 
                                                       test_size=0.1)

In [29]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [37]:
BATCH_SIZE = 8

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [38]:
sentences = test['review']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [39]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [40]:
torch.cuda.empty_cache()
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [41]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 2

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [42]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [43]:
# 재현을 위해 랜덤시드 고정
seed_val = 1
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(tqdm(train_dataloader, desc='[Train] ')):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in tqdm(validation_dataloader, desc="[Validation]"):
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...


  Batch   500  of  1,688.    Elapsed: 0:07:04.
  Batch 1,000  of  1,688.    Elapsed: 0:14:07.
  Batch 1,500  of  1,688.    Elapsed: 0:21:10.


  Average training loss: 0.72
  Training epcoh took: 0:23:48

Running Validation...



  Accuracy: 0.77
  Validation took: 0:00:56

======== Epoch 2 / 2 ========
Training...


  Batch   500  of  1,688.    Elapsed: 0:07:03.
  Batch 1,000  of  1,688.    Elapsed: 0:14:06.
  Batch 1,500  of  1,688.    Elapsed: 0:21:09.


  Average training loss: 0.46
  Training epcoh took: 0:23:48

Running Validation...



  Accuracy: 0.81
  Validation took: 0:00:56

Training complete!


In [45]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(tqdm(test_dataloader, desc='[step]')):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    625.    Elapsed: 0:00:28.
  Batch   200  of    625.    Elapsed: 0:00:56.
  Batch   300  of    625.    Elapsed: 0:01:25.
  Batch   400  of    625.    Elapsed: 0:01:54.
  Batch   500  of    625.    Elapsed: 0:02:24.
  Batch   600  of    625.    Elapsed: 0:02:54.


Accuracy: 0.81
Test took: 0:03:01


In [ ]:
# sentences = test['review']
# sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
# labels = test['label'].values

# tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# attention_masks = []
# for seq in input_ids:
#     seq_mask = [float(i>0) for i in seq]
#     attention_masks.append(seq_mask)

# test_inputs = torch.tensor(input_ids)
# test_labels = torch.tensor(labels)
# test_masks = torch.tensor(attention_masks)

# test_data = TensorDataset(test_inputs, test_masks, test_labels)
# test_sampler = RandomSampler(test_data)
# test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [71]:
for step, batch in enumerate(tqdm(test_dataloader, desc='[step]')):
    print(step)
    print(len(batch))
    print(batch[0].shape)
    print(batch[1].shape)
    print(batch[2].shape)
    print(batch[0])
    break

0
3
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8])
tensor([[  101, 59232, 92887,  ...,     0,     0,     0],
        [  101,   177, 15362,  ...,     0,     0,     0],
        [  101, 85530, 45545,  ...,     0,     0,     0],
        ...,
        [  101, 39709, 13503,  ...,     0,     0,     0],
        [  101,   177, 11850,  ...,     0,     0,     0],
        [  101, 10114, 10911,  ...,     0,     0,     0]])


In [83]:
def evaluate(text):
    model.eval()

    m = {
        0: 'female',
        1: 'unisex',
        2: 'male'
    }

    sentence = text
    sentences = ["[CLS] " + sentence + " [SEP]"]

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    test_inputs = torch.tensor(input_ids).to(device)
    test_masks = torch.tensor(attention_masks).to(device)

    outputs = model(test_inputs, token_type_ids=None, attention_mask=test_masks)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    print('this text predicted :', m[logits.argmax(1)[0]])

evaluate('hello my name is')

this text predicted : female


In [99]:
df[['gender','review']].iloc[60000].values

array(['male',
       'Timeless. A well rounded fragrance suitable for anytime , any climate. Everyone enjoys the pure simplicity and performance. Citrus, vetever and an earthy mineral smell.'],
      dtype=object)

In [100]:
evaluate('Timeless. A well rounded fragrance suitable for anytime , any climate. Everyone enjoys the pure simplicity and performance. Citrus, vetever and an earthy mineral smell.')

this text predicted : male
